## Clonando o repositório do projeto

Se você chegou até aqui e está desenvolvendo no ambiente do Google Colab, execute a célula abaixo para clonar o repositório no ambiente do Google: 

In [ ]:
!git clone https://github.com/observatorio-do-amanha-ac/sandbox.git

Ao lado, clicando no ícone que representa uma pasta, você deve ver o diretório `sandbox` contendo todos os arquivos do repositório. 

Parabéns! Agora podemos começar! :D

### Instalação das bibliotecas externas

Quem optar por desenvolver localmente, ou seja, em um notebook rodando na sua própria máquina, não se esqueça de instalar as seguintes bibliotecas conforme descrito no README do repositório: 

- pandas (pip install pandas)
- faker (pip install faker)
- duckdb (pip install duckdb)
- ipython_sql (pip install ipython_sql)

Para as pessoas que optarem pelo desenvolvimento integrado ao Google Colab, deve executar as células abaixo (o pandas já está instalado por padrão no ambiente do Google):


In [ ]:
!pip install faker

In [ ]:
!pip install duckdb

## Importando as bibliotecas que iremos utilizar

In [1]:
import os
from datetime import date, datetime
import uuid
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, text
import random
from faker import Faker


## Definindo o caminho do banco de dados

### Para desenvolvimento em um notebook local

Caso você opte por clonar o repositório para desenvolver localmente, utilize o código abaixo para garantir o caminho exato do arquivo `sqlite`:

In [2]:
base_path = os.path.dirname(os.getcwd())
database_dir = os.path.join(base_path, "databases")

db_oltp_file = "sqlite_oltp_ac_fake_database.db"
db_olap_file =  "duckdb_olap_ac_fake_database.duckdb"

db_oltp_path = os.path.join(database_dir, db_oltp_file) 
db_olap_path = os.path.join(database_dir, db_olap_file)


### Para desenvolver no ambiente do Google Colab

Caso você queira utilizar o ambiente do Google Colaba para desenvolvimento, utilize o caminho do arquivo de dos bancos de dados seguindo clicando nos "3 pontinhos" ao lado direito dos arquivos na pasta `datasets` e escolhendo a opção "Copiar Caminho":

In [ ]:
# Define os caminhos do banco de dados no Colab
db_oltp_path = "/content/sandbox/databases/sqlite_oltp_ac_fake_database.db"
db_olap_path = "/content/sandbox/databases/duckdb_olap_ac_fake_database.duckdb"

## Restaura arquivo banco de dados

Essa função pode ser utilizada para restaurar o banco do zero caso você encontre algum problema... vai que, né?

In [3]:
# Esta função restaura o arquivo de banco de dados
def db_restore(db_path):
    
    # Verifica se o arquivo existe
    if os.path.exists(db_path):
        # Remove o arquivo existente
        os.remove(db_path)
        print(f"Arquivo {db_path} removido.")
    else:
        print(f"Arquivo {db_path} não encontrado.")
    
    # Cria um novo arquivo SQLite
    with open(db_path, 'w') as file:
        pass  # Não adiciona nada ao arquivo
    
    print(f"Arquivo {db_path} criado ou substituído.")
    

In [4]:
db_restore(db_oltp_path) 

Arquivo /home/tiago/workspace/repos/observatorio-do-amanha/sandbox/databases/sqlite_oltp_ac_fake_database.db removido.
Arquivo /home/tiago/workspace/repos/observatorio-do-amanha/sandbox/databases/sqlite_oltp_ac_fake_database.db criado ou substituído.


## Abre a conexão com os bancos de dados

In [5]:
# Conexão ao banco de dados SQLite
sqlite_conn = sqlite3.connect(db_oltp_path)
cursor = sqlite_conn.cursor()

In [ ]:
# Conexão ao banco de dados DuckDB 
duckdb_conn = duckdb.connect(db_olap_path)

## Define o Schema dobanco de dados transacional

In [6]:
# Script para criar as tabelas
schema_script = """
CREATE TABLE IF NOT EXISTS CAD_PESSOA (
    ID_PESSOA TEXT UNIQUE PRIMARY KEY,
    NOME VARCHAR(100) NOT NULL,
    CPF VARCHAR(11) UNIQUE NOT NULL,
    DATA_NASC DATE NOT NULL,
    GENERO VARCHAR(20),
    ORIENTACAO_SEXUAL VARCHAR(20),
    RACA_ETNIA VARCHAR(50),
    ESCOLARIDADE VARCHAR(50),
    ESTADO_CIVIL VARCHAR(20),
    FAIXA_DE_RENDA VARCHAR(50),
    AREA_DE_ATUACAO VARCHAR(100),
    TELEFONE VARCHAR(15),
    EMAIL VARCHAR(100) UNIQUE,
    URL_REDE_SOCIAL VARCHAR(255),
    ENDERECO VARCHAR(255),
    CEP VARCHAR(10),
    LOGRADOURO VARCHAR(100),
    NUMERO VARCHAR(10),
    BAIRRO VARCHAR(100),
    CIDADE VARCHAR(100),
    ESTADO VARCHAR(2),
    PAIS VARCHAR(100),
    DATA_VISITA DATE
);

CREATE TABLE IF NOT EXISTS ONU_ODS (
    ID_ODS INTEGER PRIMARY KEY AUTOINCREMENT,
    NOME_ODS VARCHAR(100) NOT NULL
);

CREATE TABLE IF NOT EXISTS DOMINIOS (
    ID_DOMINIO INTEGER PRIMARY KEY AUTOINCREMENT,
    NOME_DOMINIO VARCHAR(100) NOT NULL,
    RESPONSAVEL_DOMINIO VARCHAR(100) NOT NULL
);

CREATE TABLE IF NOT EXISTS SERVICOS (
    ID_SERVICO INTEGER PRIMARY KEY AUTOINCREMENT,
    TITULO_SERVICO VARCHAR(100) NOT NULL
);

CREATE TABLE IF NOT EXISTS PROGRAMAS (
    ID_PROGRAMA TEXT UNIQUE PRIMARY KEY,
    ID_DOMINIO INTEGER NOT NULL,
    TITULO_PROGRAMA VARCHAR(100) NOT NULL,
    FOREIGN KEY (ID_DOMINIO) REFERENCES DOMINIOS(ID_DOMINIO)
);

CREATE TABLE IF NOT EXISTS PROGRAMA_ODS (
    ID_PROGRAMA TEXT NOT NULL,
    ID_ODS INTEGER NOT NULL,
    PRIMARY KEY (ID_PROGRAMA, ID_ODS),
    FOREIGN KEY (ID_PROGRAMA) REFERENCES PROGRAMAS(ID_PROGRAMA),
    FOREIGN KEY (ID_ODS) REFERENCES ONU_ODS(ID_ODS)
);

CREATE TABLE IF NOT EXISTS INSCRICOES (
    ID_INSCRICAO TEXT UNIQUE PRIMARY KEY,
    ID_PROGRAMA TEXT NOT NULL,
    ID_PESSOA TEXT NOT NULL,
    DATA_INSCRICAO DATE NOT NULL,
    FOREIGN KEY (ID_PROGRAMA) REFERENCES PROGRAMAS(ID_PROGRAMA),
    FOREIGN KEY (ID_PESSOA) REFERENCES CAD_PESSOA(ID_PESSOA)
);

CREATE TABLE IF NOT EXISTS PARTICIPANTE_PROGRAMA (
    ID_PROGRAMA TEXT NOT NULL,
    ID_INSCRICAO TEXT NOT NULL,
    PRIMARY KEY (ID_PROGRAMA, ID_INSCRICAO),
    FOREIGN KEY (ID_PROGRAMA) REFERENCES PROGRAMAS(ID_PROGRAMA),
    FOREIGN KEY (ID_INSCRICAO) REFERENCES INSCRICOES(ID_INSCRICAO)
);

CREATE TABLE IF NOT EXISTS AGENDAMENTO_SERVICO (
    ID_AGENDAMENTO INTEGER PRIMARY KEY AUTOINCREMENT,
    ID_PESSOA TEXT NOT NULL,
    ID_SERVICO INTEGER NOT NULL,
    DATETIME_AGENDAMENTO DATETIME NOT NULL,
    FOREIGN KEY (ID_PESSOA) REFERENCES CAD_PESSOA(ID_PESSOA),
    FOREIGN KEY (ID_SERVICO) REFERENCES SERVICOS(ID_SERVICO)
);
"""

# Executar o script SQL para criar as tabelas
cursor.executescript(schema_script)

# Confirmar as alterações e fechar a conexão
sqlite_conn.commit()

print("Tabelas criadas com sucesso!")

Tabelas criadas com sucesso!


In [7]:
# Consultar as tabelas existentes
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Recuperar e imprimir os nomes das tabelas
tables = cursor.fetchall()
print("Tabelas no banco de dados:")
for table in tables:
    print(table[0])


Tabelas no banco de dados:
CAD_PESSOA
ONU_ODS
sqlite_sequence
DOMINIOS
SERVICOS
PROGRAMAS
PROGRAMA_ODS
INSCRICOES
PARTICIPANTE_PROGRAMA
AGENDAMENTO_SERVICO


## CAD_PESSOA

In [8]:
fake = Faker('pt_BR')
# Cria uma quantidade determinada de registros aleatórios no banco sqlite
def insert_into_cad_pessoa(qtd_registros=10):
    bairros = [
        "Agamenon Magalhães", "Alto do Moura", "Boa Vista", "Caiucá", "Cedro", "Centro", "Cidade Jardim",
        "Cohab I", "Cohab II", "Divinópolis", "Indianópolis", "Jardim Panorama", "João Mota", "Lagoa do Algodão",
        "Maurício de Nassau", "Monte Bom Jesus", "Nova Caruaru", "Petrópolis", "Pinheirópolis", "Salgado",
        "Rendeiras", "Santa Clara", "São Francisco", "São José", "São João da Escócia", "Universitário",
        "Vassoural", "Vila Kennedy", "Xique xique"
    ]
    emails_gerados = set()  # Conjunto para garantir unicidade dos e-mails
    cpfs_gerados = set()  # Conjunto para garantir unicidade dos CPFs

    for _ in range(qtd_registros):
        id_pessoa = str(uuid.uuid4())
        nome = fake.name()

        # Garantir que o CPF seja único
        cpf = fake.cpf()
        while cpf in cpfs_gerados:
            cpf = fake.cpf()
        cpfs_gerados.add(cpf)

        data_nasc = fake.date_of_birth(minimum_age=18, maximum_age=90).strftime('%Y-%m-%d')
        genero = random.choice(['Masculino', 'Feminino', 'Outro'])
        orientacao_sexual = random.choice(['Heterossexual', 'Homossexual', 'Bissexual', 'Outro'])
        raca_etnia = random.choice(['Branca', 'Preta', 'Parda', 'Amarela', 'Indígena'])
        escolaridade = random.choice(['Ensino Fundamental', 'Ensino Médio', 'Ensino Superior', 'Pós-graduação'])
        estado_civil = random.choice(['Solteiro', 'Casado', 'Divorciado', 'Viúvo'])
        faixa_de_renda = random.choice(['Até 1 salário mínimo', '1 a 3 salários mínimos', '3 a 5 salários mínimos', 'Mais de 5 salários mínimos'])
        area_de_atuacao = random.choice(['Tecnologia', 'Educação', 'Saúde', 'Comércio', 'Indústria', 'Serviços'])
        telefone = fake.phone_number()
        
        # Garantir que o e-mail seja único
        email = fake.email()
        while email in emails_gerados:
            email = fake.email()
        emails_gerados.add(email)

        url_rede_social = fake.url()
        endereco = fake.street_address()
        cep = fake.postcode()
        logradouro = fake.street_name()
        numero = str(fake.building_number())
        bairro = random.choice(bairros)
        cidade = 'Caruaru'
        estado = 'PE'
        pais = 'Brasil'
        data_visita = fake.date_this_year().strftime('%Y-%m-%d')

        # Inserir os dados na tabela CAD_PESSOA
        cursor.execute('''
            INSERT INTO CAD_PESSOA (
                ID_PESSOA, NOME, CPF, DATA_NASC, GENERO, ORIENTACAO_SEXUAL, RACA_ETNIA, ESCOLARIDADE, ESTADO_CIVIL,
                FAIXA_DE_RENDA, AREA_DE_ATUACAO, TELEFONE, EMAIL, URL_REDE_SOCIAL, ENDERECO, CEP, LOGRADOURO,
                NUMERO, BAIRRO, CIDADE, ESTADO, PAIS, DATA_VISITA
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (id_pessoa, nome, cpf, data_nasc, genero, orientacao_sexual, raca_etnia, escolaridade, estado_civil, faixa_de_renda,
              area_de_atuacao, telefone, email, url_rede_social, endereco, cep, logradouro, numero, bairro, cidade, estado,
              pais, data_visita))

    # Confirmar as alterações
    sqlite_conn.commit()

# Inserir 1000 registros fictícios na tabela CAD_PESSOA
insert_into_cad_pessoa(1000)



## ONU_ODS

In [9]:
ods = [
    "Erradicação da pobreza",
    "Fome zero e agricultura sustentável",
    "Saúde e bem-estar",
    "Educação de qualidade",
    "Igualdade de gênero",
    "Água potável e saneamento",
    "Energia limpa e sustentável",
    "Trabalho decente e inclusão social",
    "Indústria, inovação e infraestrutura",
    "Redução das desigualdades",
    "Cidades e comunidades sustentáveis",
    "Consumo e produção sustentáveis",
    "Ação contra mudança global do clima",
    "Vida na água",
    "Vida terrestre",
    "Paz, justiça e participação humanitária",
    "Parcerias e meios de implementação",
]

def insert_into_onu_ods(ods_list):

    for ods in ods_list:
        cursor.execute('''
            INSERT INTO ONU_ODS (NOME_ODS)
            VALUES (?)
        ''', (ods,))

    sqlite_conn.commit()

insert_into_onu_ods(ods)


## DOMINIOS

In [10]:
# Dicionário que mapeia cada domínio ao seu respectivo responsável
mapa_dominio_responsavel = {
    "Gestão": "Pâmela",
    "Inovação Aberta": "Lais",
    "Equidade de gênero": "Gabi",
    "Empreendedorismo": "Mirelli",
    "Suporte em TI": "Lucas",
    "Comunicação e Marketing": "Bono"
}

def inserir_into_dominios(map):


    nomes_dominios = list(map.keys())
    responsaveis_dominio = list(map.values())
    qtd_dominios = len(nomes_dominios)

    for i in range(qtd_dominios):

        # Selecionar o responsável correspondente ao domínio
        responsavel_dominio = responsaveis_dominio[i]
        nome_dominio = nomes_dominios[i]

        cursor.execute('''
            INSERT INTO DOMINIOS (NOME_DOMINIO, RESPONSAVEL_DOMINIO)
            VALUES (?, ?)
        ''', (nome_dominio, responsavel_dominio))

    sqlite_conn.commit()


inserir_into_dominios(mapa_dominio_responsavel)

## SERVICOS

In [11]:
servicos = [
    "Coworking",
    "Laboratórios de Criação e Prototipagem",
    "Capacitação e Formação",
    "Apoio a Startups",
    "Eventos e Networking",
]

def insert_into_servicos(lista_servicos):

    for servico in lista_servicos:
        cursor.execute('''
            INSERT INTO SERVICOS (TITULO_SERVICO)
            VALUES (?)
        ''', (servico,))

    sqlite_conn.commit()

insert_into_servicos(servicos)

## PROGRAMAS

In [12]:
programas_dominios = {
    "Clubes de Programação": 3,
    "Elas que Lideram": 3,
    "Mulheres no Alto": 3,
    "Minas Data Lab": 3,
    "Transforma": 3,
    "Minas na Produção": 3,
    "Agreste Futurista": 3,
    "Observatório do amanhã":2,
    "Desafios de sustentabilidade": 2,
    "MMA (moda x música x audiovisual)": 2,
    ".GOV + Dotlab": 2,
    "Ciclo de inovação aberta": 2,
    "Incubação":4,
    "Mind the Bizz": 4,
    "Agreste Moda": 4,
}

def insert_into_programa():

    titulos_programas = list(programas_dominios.keys())
    dominios_programas = list(programas_dominios.values())
    qtd_programas = len(titulos_programas)

    for i in range(qtd_programas):

        id_programa = str(uuid.uuid4())
        titulo_programa = titulos_programas[i]
        id_dominio_programa = dominios_programas[i]
        cursor.execute('''
            INSERT INTO PROGRAMAS (ID_PROGRAMA, ID_DOMINIO, TITULO_PROGRAMA)
            VALUES (?, ?, ?)
        ''', (id_programa, id_dominio_programa, titulo_programa))

    sqlite_conn.commit()

insert_into_programa()

## PROGRAMA_ODS

In [ ]:
## Todo

## INSCRICOES

In [13]:
def insert_into_inscricoes_programas(qtd_registros=10):
    cursor.execute("SELECT ID_PESSOA FROM CAD_PESSOA")
    pessoas = cursor.fetchall()
    cursor.execute("SELECT ID_PROGRAMA FROM PROGRAMAS")
    programas = cursor.fetchall()

    # Define o intervalo de datas
    data_inicio = date(2024, 1, 15)
    data_fim = date(2024, 8, 1)

    for _ in range(qtd_registros):
        id_inscricao = str(uuid.uuid4())
        id_programa = random.choice(programas)[0]
        id_pessoa = random.choice(pessoas)[0]
        data_inscricao = fake.date_between(start_date=data_inicio, end_date=data_fim).strftime('%Y-%m-%d')

        cursor.execute('''
            INSERT INTO INSCRICOES (ID_INSCRICAO, ID_PROGRAMA, ID_PESSOA, DATA_INSCRICAO)
            VALUES (?, ?, ?, ?)
        ''', (id_inscricao, id_programa, id_pessoa, data_inscricao))

    sqlite_conn.commit()

insert_into_inscricoes_programas(350)

## PARTICIPANTE_PROGRAMA

In [14]:
def insert_into_programas_participante(qtd_registros=10):
    cursor.execute("SELECT ID_PROGRAMA FROM PROGRAMAS")
    programas = cursor.fetchall()
    cursor.execute("SELECT ID_INSCRICAO FROM INSCRICOES")
    inscricoes = cursor.fetchall()

    for _ in range(qtd_registros):
        id_inscricao = random.choice(inscricoes)[0]
        id_programa = random.choice(programas)[0]

        # Verifica se a combinação já existe
        cursor.execute('''
            SELECT COUNT(*) FROM PARTICIPANTE_PROGRAMA
            WHERE ID_PROGRAMA = ? AND ID_INSCRICAO = ?
        ''', (id_programa, id_inscricao))

        if cursor.fetchone()[0] == 0:
            cursor.execute('''
                INSERT INTO PARTICIPANTE_PROGRAMA (ID_PROGRAMA, ID_INSCRICAO)
                VALUES (?, ?)
            ''', (id_programa, id_inscricao))

    sqlite_conn.commit()

insert_into_programas_participante(250)

## AGENDAMENTO_SERVICO

In [15]:
def insert_into_agendamento_servico(qtd_registros=10):
    cursor.execute("SELECT ID_SERVICO FROM SERVICOS")
    servicos = cursor.fetchall()
    cursor.execute("SELECT ID_PESSOA FROM CAD_PESSOA")
    pessoas = cursor.fetchall()

     # Define o intervalo de datas
    data_inicio = date(2024, 1, 15)
    data_fim = date(2024, 8, 1)

    for _ in range(qtd_registros):
        # id_agendamento = str(uuid.uuid4()) um dia talvez eu precise dessa variável
        id_servico = random.choice(servicos)[0]
        id_pessoa = random.choice(pessoas)[0]
        datetime_agendamento = fake.date_between(start_date=data_inicio, end_date=data_fim).strftime('%Y-%m-%d %H:%M:%S')

        cursor.execute('''
            INSERT INTO AGENDAMENTO_SERVICO (ID_SERVICO, ID_PESSOA, DATETIME_AGENDAMENTO)
            VALUES (?, ?, ?)
        ''', (id_servico, id_pessoa, datetime_agendamento))

    sqlite_conn.commit()

insert_into_agendamento_servico(500)

## Consultando o banco de dodos com python

**Assim:**

In [ ]:
query = """
SELECT COUNT(*)
FROM CAD_PESSOA
"""
cursor.execute(query)
result = cursor.fetchall()

for row in result:
    print(row)


**Também assim:**

In [ ]:
query = """
SELECT *
FROM CAD_PESSOA
"""
cursor.execute(query)
result = cursor.fetchall()

# Instanciando um Objeto DataFrame:
df_cad_pessoa = pd.DataFrame(result, columns=[column[0] for column in cursor.description])
df_cad_pessoa.head()

**Ou assim:**

In [ ]:
query = """
SELECT *
FROM CAD_PESSOA
"""
# Utilizando o método read_sql_query do pandas:
df_cad_pessoa = pd.read_sql_query(query, conn)
df_cad_pessoa.head()

In [ ]:
sqlite_conn.close()

## Consultando o Banco utilizando SQL diretamente

In [16]:
%load_ext sql
%sql sqlite:///{db_oltp_path}

In [17]:
%%sql
SELECT *
FROM CAD_PESSOA
LIMIT 5

 * sqlite:////home/tiago/workspace/repos/observatorio-do-amanha/sandbox/databases/sqlite_oltp_ac_fake_database.db
Done.


ID_PESSOA,NOME,CPF,DATA_NASC,GENERO,ORIENTACAO_SEXUAL,RACA_ETNIA,ESCOLARIDADE,ESTADO_CIVIL,FAIXA_DE_RENDA,AREA_DE_ATUACAO,TELEFONE,EMAIL,URL_REDE_SOCIAL,ENDERECO,CEP,LOGRADOURO,NUMERO,BAIRRO,CIDADE,ESTADO,PAIS,DATA_VISITA
a3a922ff-6f01-4212-a713-e0cd3cafbac9,Erick Sousa,189.352.674-73,1947-01-15,Feminino,Outro,Amarela,Ensino Médio,Viúvo,Mais de 5 salários mínimos,Comércio,+55 (051) 6687 4362,yrocha@example.org,http://www.rodrigues.br/,"Ladeira Júlia da Rosa, 30",93829-322,Área Cassiano,707,Agamenon Magalhães,Caruaru,PE,Brasil,2024-08-20
93400d77-f64d-4c0c-89b6-ffc4dbaab85c,Dr. Ravy Novais,526.407.839-47,1945-07-15,Feminino,Heterossexual,Preta,Ensino Superior,Viúvo,3 a 5 salários mínimos,Comércio,+55 (081) 2823 1474,ana-clara64@example.net,https://www.duarte.com/,Residencial Gustavo Henrique Dias,96309710,Área Gael Henrique da Cruz,8,Cidade Jardim,Caruaru,PE,Brasil,2024-07-09
f191d203-ec67-4f20-8519-c7295a8f4b46,Rafael da Conceição,325.716.984-19,1961-03-11,Feminino,Heterossexual,Branca,Ensino Fundamental,Divorciado,3 a 5 salários mínimos,Educação,61 7153-7983,maria-cecilia24@example.net,https://cunha.br/,"Núcleo Santos, 5",05128450,Parque de Silveira,89,Jardim Panorama,Caruaru,PE,Brasil,2024-07-10
c22d3d4b-fe82-4e54-98aa-d02bc94266f8,Carolina Freitas,703.492.586-00,1990-11-08,Outro,Bissexual,Preta,Ensino Superior,Solteiro,3 a 5 salários mínimos,Tecnologia,61 0922 5803,elisasousa@example.org,http://www.gomes.com/,"Quadra Lucas Gabriel Cassiano, 57",74720-264,Praia de Mendes,73,Petrópolis,Caruaru,PE,Brasil,2024-06-04
c2279704-3e32-4c59-9a4d-cfc8bbdc1e14,Ravi Barbosa,431.270.956-43,2002-12-29,Outro,Heterossexual,Parda,Pós-graduação,Divorciado,Mais de 5 salários mínimos,Serviços,+55 (041) 3518-4019,maria-fernandacavalcanti@example.com,https://porto.com/,"Aeroporto Igor Barros, 9",52509460,Trevo Pires,81,Indianópolis,Caruaru,PE,Brasil,2024-06-01


## Misturando as duas formas de consultar os dados

In [ ]:
cad_pessoa_df = %sql SELECT * FROM CAD_PESSOA
cad_pessoa_df = cad_pessoa_df.DataFrame()
cad_pessoa_df.head()

## Entendendo SQL e exercitando o cérebro

Vamos rever alguns conceitos básicos extremamente úteis e exercitarmos para nos familiarizarmos cada vez mais com SQL.  

1. Cláusula SELECT e FROM

A cláusula SELECT é usada para especificar quais colunas de dados você deseja recuperar de um banco de dados. Já a cláusula FROM serve para indicar a tabela ou as tabelas de onde os dados serão extraídos.

Exemplo:

In [19]:
%%sql
SELECT 
    NOME
   ,CPF 
FROM CAD_PESSOA 
LIMIT 5

 * sqlite:////home/tiago/workspace/repos/observatorio-do-amanha/sandbox/databases/sqlite_oltp_ac_fake_database.db
Done.


NOME,CPF
Erick Sousa,189.352.674-73
Dr. Ravy Novais,526.407.839-47
Rafael da Conceição,325.716.984-19
Carolina Freitas,703.492.586-00
Ravi Barbosa,431.270.956-43


Acima, selecionamos as colunas NOME e CPF da tabela CAD_PESSOA.

2. Cláusula WHERE

A cláusula WHERE é usada para filtrar registros com base em uma condição específica. Somente as linhas que atendem à condição serão retornadas.

Exemplo:

In [22]:
%%sql
SELECT 
    NOME
   ,ESCOLARIDADE
FROM CAD_PESSOA
WHERE CPF = "703.492.586-00"

 * sqlite:////home/tiago/workspace/repos/observatorio-do-amanha/sandbox/databases/sqlite_oltp_ac_fake_database.db
Done.


NOME,ESCOLARIDADE
Carolina Freitas,Ensino Superior


O exemplo acima retorna NOME e ESCOLARIDADE de quando o CPF "703.492.586-00" for encontrado no banco. Lembre-se sempre, o nosso código cria um banco de dados com registros aleatórios, ou seja, toda vez que executamos a criaçã de um banco dados novos são criado, portanto, é provavel que o CPF utilizado no exemplo não exista na sua base de dados, ok?

4. Cláusula JOIN

A cláusula JOIN é usada para combinar registros de duas ou mais tabelas com base em uma condição relacionada entre elas.

Existem diferentes tipos de JOIN:

- **INNER JOIN**: Retorna os registros que têm correspondência em ambas as tabelas.
- **LEFT JOIN**: Retorna todos os registros da tabela à esquerda e os registros correspondentes da tabela à direita. Se não houver correspondência, o resultado da tabela à direita será NULL.
- **RIGHT JOIN**: Retorna todos os registros da tabela à direita e os registros correspondentes da tabela à esquerda. Se não houver correspondência, o resultado da tabela à esquerda será NULL.
**FULL OUTER JOIN**: Retorna todos os registros quando há uma correspondência em uma das tabelas.

Por enquanto vamos utilizar apenas o INNER JOIN, que também pode ser usado apenas com a palavra-chave JOIN.

Os conceito acima, são muitos semelhantes à Teoria dos Conjuntos da matemática. 

Exemplo:

In [33]:
%%sql
SELECT 
    C.ID_PESSOA
   ,C.CPF
   ,I.DATA_INSCRICAO
   ,P.TITULO_PROGRAMA
FROM CAD_PESSOA AS C
JOIN INSCRICOES AS I
ON C.ID_PESSOA = I.ID_PESSOA
JOIN PARTICIPANTE_PROGRAMA AS PP
ON I.ID_INSCRICAO = PP.ID_INSCRICAO
JOIN PROGRAMAS AS P 
ON PP.ID_PROGRAMA = P.ID_PROGRAMA
LIMIT 5

 * sqlite:////home/tiago/workspace/repos/observatorio-do-amanha/sandbox/databases/sqlite_oltp_ac_fake_database.db
Done.


ID_PESSOA,CPF,DATA_INSCRICAO,TITULO_PROGRAMA
2c985750-095c-44b1-9b88-51ccdf7d68d8,608.175.492-85,2024-02-02,Mulheres no Alto
2ad1d785-6abf-45d5-a3ed-29582e1cd83a,385.249.106-15,2024-01-23,Observatório do amanhã
b6ce4072-8356-47b3-a0ad-221498783afe,815.342.076-35,2024-02-27,Mind the Bizz
3fbff1aa-0043-4e22-998f-df56880cb51a,673.421.095-25,2024-05-18,Ciclo de inovação aberta
d73442ef-e493-48eb-8a2c-675a74c2e628,126.759.380-68,2024-03-04,Desafios de sustentabilidade


No exemplo acima, eu seleciono os registros que asassociam pessoas cadastradas em CAD_PESSOA aos programas em que estão inscritas. A consulta retorna o ID_PESSOA, o CPF da pessoa, a DATA_INSCRICAO no programa, e o TITULO_PROGRAMA. Para isso, a query faz junções entre as tabelas CAD_PESSOA, INSCRICOES, PARTICIPANTE_PROGRAMA e PROGRAMAS, conectando as inscrições de pessoas aos respectivos programas em que participaram.

## Exercícios!!

1. Liste o nome das pessoas e o nome dos domínios relacionados aos programas nos quais elas estão inscritas, onde o domínio é "Inovação Aberta".

2. Selecione o nome das pessoas que fizeram agendamentos de serviços nos últimos 30 dias.

3. Liste o nome das pessoas participantes de programas e seus respectivos bairros.